### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [9]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [10]:
X_test.columns

Index(['AccountWeeks', 'ContractRenewal', 'DataPlan', 'DataUsage',
       'CustServCalls', 'DayMins', 'DayCalls', 'MonthlyCharge', 'OverageFee',
       'RoamMins'],
      dtype='object')

In [13]:
import urllib.request
import json      

def get_prediction(x):
    accountweeks, contractrenewal, dataplan, datausage, custservcalls, daymins, daycalls, monthlycharge, overagefee, roammins = x
    body = {'AccountWeeks': accountweeks, 
            'ContractRenewal': contractrenewal, 
            'DataPlan': dataplan, 
            'DataUsage': datausage,
            'CustServCalls': custservcalls, 
            'DayMins': daymins, 
            'DayCalls': daycalls, 
            'MonthlyCharge': monthlycharge, 
            'OverageFee': overagefee,
            'RoamMins': roammins} 

    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [19]:
test_one = X_test.iloc[:1]
print(test_one)
get_prediction(test_one)

   AccountWeeks  ContractRenewal  DataPlan  DataUsage  CustServCalls  DayMins  \
0           160                1         0        0.0              3    256.0   

   DayCalls  MonthlyCharge  OverageFee  RoamMins  
0       111           60.0        9.37      11.5  


URLError: <urlopen error [Errno 61] Connection refused>

In [14]:
%%time
predictions = X_test.iloc[:100].apply(lambda x: get_prediction(x), 1)

URLError: <urlopen error [Errno 61] Connection refused>

In [8]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:500])

0.9803161348046525

In [7]:
#X_test[['description', 'company_profile', 'benefits']].head(3)

Давайте подумаем, где наше api может сломаться

In [9]:
get_prediction("", "", None)

0.13518841848029955